# Flask

In [ ]:
%%writefile simple_flask.py

from flask import Flask
app = Flask(__name__)
 
@app.route("/")
def hello():
    return "Hello World!"
 
if __name__ == "__main__":
    app.run()
    
#Теперь нужно запустить, через python simple_flask.py

In [ ]:
%%writefile  simple_flask.py

from flask import Flask
app = Flask(__name__)
 
@app.route("/")
def hello():
    return "Hello World!"

@app.route("/members/<string:name>/")
def getMember(name):
    return name

if __name__ == "__main__":
    app.run()
#Теперь нужно запустить, через python simple_flask.py

# Парсинг

In [1]:
!pip install requests bs4 user_agent selenium

    100% |████████████████████████████████| 911kB 741kB/s 
    100% |████████████████████████████████| 122kB 788kB/s 
  Running setup.py bdist_wheel for bs4 ... - \ done
  Stored in directory: /home/elizabeth/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Running setup.py bdist_wheel for user-agent ... - \ done
  Stored in directory: /home/elizabeth/.cache/pip/wheels/92/80/3f/5d79277825042f2d4d447f594e3fc046f1e506f2b481d364b2
Successfully built bs4 user-agent
You are using pip version 8.1.2, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [3]:
#Сделаем простой запрос, получим страничку с ленты
r = requests.get("https://lenta.ru/2018/06/16/")
print(r.ok)
print(r.status_code)
print(r.cookies)

True
200
<RequestsCookieJar[<Cookie lid=vAsAAJmO0FuSRkUSAV44GgB= for .lenta.ru/>, <Cookie lids=4824469223A652FE for .lenta.ru/>]>


In [ ]:
print(r.text)

In [4]:
#Попробуем запросить несуществующую страницу
r = requests.get("https://lenta.ru/2018/10/32/")
print(r.ok)
print(r.status_code)
print(r.cookies)

False
404
<RequestsCookieJar[<Cookie lid=vAsAAOqO0FvTZkwHAcbKGAB= for .lenta.ru/>]>


In [5]:
#TODO загрузите основную страницу техноатома
r = requests.get("https://atom.mail.ru/feed/")
print(r.ok)
print(r.status_code)
print(r.cookies)

True
200
<RequestsCookieJar[<Cookie csrftoken=xOdSeXsx06pLOrsxRQkcVcm3v0LTgYZs for atom.mail.ru/>]>


In [11]:
#Создаем элемент типа soup
r = requests.get("https://lenta.ru/2018/06/16/")
soup = BeautifulSoup(r.text, "html.parser")

In [7]:
#Посмотрим на атрибуты
print(soup.attrs)

{}


In [8]:
#Найдем элементы в теге div. При этом будут найдены все теги, любой вложенности
print(soup.find_all('div')[0])

<div><img alt="" src="https://mc.yandex.ru/watch/46178820" style="position:absolute; left:-9999px;"/></div>


In [9]:
print(soup.find_all('div')[0].attrs)

{}


In [12]:
#Найдем все элементы под тегом div, у которых атрибут класс:titles
soup.findAll("div", attrs={"class": ["titles"]})

[<div class="titles"><h3><a href="/articles/2018/06/16/spain_portugal/"><span>Чистый кайф</span></a></h3><div class="rightcol">Шикарный Роналду остановил Испанию: такого футбола Россия еще не видела</div></div>,
 <div class="titles"><h3><a href="/articles/2018/06/16/cold_war/"><span>«Они будут в нас стрелять»</span></a></h3><div class="rightcol">Почему СССР тайно дружил с Африкой и избежал войны с Китаем</div></div>,
 <div class="titles"><h3><a href="/photo/2018/06/16/macedonia/"><span>Славяне не нужны</span></a></h3><div class="rightcol">Эту страну лишили названия. Скоро там останутся одни мусульмане</div></div>,
 <div class="titles"><h3><a href="/articles/2018/06/16/barron/"><span>Плодитесь и размножайтесь</span></a></h3><div class="rightcol">Он 30 лет колесил по миру, соблазнял женщин и тайно породил десятки детей</div></div>,
 <div class="titles"><h3><a href="/articles/2018/06/16/kino16062018/"><span>Папа может</span></a></h3><div class="rightcol">Кино недели: от «Суперсемейки 2» д

In [10]:
soup.find_all('div')[0].img.attrs

{'alt': '',
 'src': 'https://mc.yandex.ru/watch/46178820',
 'style': 'position:absolute; left:-9999px;'}

In [13]:
print(soup.findAll("div", attrs={"class": ["titles"]})[0])
print()
print(soup.findAll("div", attrs={"class": ["titles"]})[0].find('a'))
print()
print(soup.findAll("div", attrs={"class": ["titles"]})[0].find('a').attrs)
print()
print(soup.findAll("div", attrs={"class": ["titles"]})[0].find('a')['href'])
print()
print(soup.findAll("div", attrs={"class": ["titles"]})[0].find('a').text)

<div class="titles"><h3><a href="/articles/2018/06/16/spain_portugal/"><span>Чистый кайф</span></a></h3><div class="rightcol">Шикарный Роналду остановил Испанию: такого футбола Россия еще не видела</div></div>

<a href="/articles/2018/06/16/spain_portugal/"><span>Чистый кайф</span></a>

{'href': '/articles/2018/06/16/spain_portugal/'}

/articles/2018/06/16/spain_portugal/

Чистый кайф


In [20]:
#TODO сохранить все ссылки в словарь вида: {href: title}
for i in soup.findAll("div", attrs={"class": ["titles"]}):
    print(i.find('a')['href'])
    print(i.find('a').text)
    

/articles/2018/06/16/spain_portugal/
Чистый кайф
/articles/2018/06/16/cold_war/
«Они будут в нас стрелять»
/photo/2018/06/16/macedonia/
Славяне не нужны
/articles/2018/06/16/barron/
Плодитесь и размножайтесь
/articles/2018/06/16/kino16062018/
Папа может
/photo/2018/06/16/hotels/
От Геленджика до Грозного
/articles/2018/06/16/the_architext/
После нас
/news/2018/06/16/ugrozi/
Судье пригрозили смертью за арест отрицателя холокоста
/articles/2018/06/16/gayropa_fest/
Рижский прайд
/news/2018/06/16/bonds/
Россия избавилась от госдолга США
/news/2018/06/16/croc/
На берег реки выползли десятки крокодилов
/news/2018/06/16/kiev_nafta/
Киев подал в суд на «Нафтогаз»
/news/2018/06/16/hamlo/
Трамп нагрубил половине лидеров G7
/news/2018/06/16/chekrym/
Госдеп США предостерег американцев от попыток забраться на Эльбрус
/news/2018/06/16/ignored/
Команда Трампа саботировала его планы встретиться с Путиным
/news/2018/06/16/chubbs/
В Калифорнии спасли растолстевшего втрое бездомного кота
/news/2018/06/16

In [23]:
def get_product_info_avito(url):
    product_page = requests.get(url)
    soup = BeautifulSoup(product_page.text, "html.parser")
    title = re.findall(r"(?=content=\"([^\"]*)\")",
                       str(soup.find("meta", attrs={"name":"mrc__share_title"})), re.UNICODE)[0]
    if title.startswith(u"Объявление на Avito -"):
        title = title[21:]
    descr = re.findall(r"(?=content=\"([^\"]*)\")",
                       str(soup.find("meta", attrs={"name":"mrc__share_description"})), re.UNICODE)[0]
    return url, title, descr

def get_smilar_avito(url):
    headers = {u"accept": u"*/*", u"accept-encoding": u"gzip, deflate, br",
           u"accept-language": u"ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
           u"cookie": u"u=26m3i302.1fc6nq6.flumvzf17n; buyer_location_id=637640; abp=1; _ga=GA1.2.1386374517.1519811174; _vwo_uuid_v2=D5AF40B1213C5F49AEE8A45D8584418ED|06e85eceee6cdc4810096a7268f54acb; cto_lwid=85c9fbb1-8918-4cc4-a91d-2a757134e844; _vis_opt_s=1%7C; _vis_opt_test_cookie=1; _ym_uid=1519811174428571120; _vwo_uuid=D5AF40B1213C5F49AEE8A45D8584418ED; __gads=ID=c16722fa56933ba1:T=1519811174:S=ALNI_MYLAba7KfXbX-Amt7ypPSVfyyTdzA; _vis_opt_exp_46_exclude=1; __utma=99926606.1386374517.1519811174.1521126280.1521126280.1; __utmc=99926606; __utmz=99926606.1521126280.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); dfp_group=52; _vwo_ds=3%3Aa_0%2Ct_0%3A0%241526386623%3A47.11855471%3A%3A%3A18_0%2C12_0; sessid=bbc4979ea3347bd02e9b3ed7349589c7.1527492578; _gid=GA1.2.91958680.1527492579; _ym_isad=1; f=5.673c10cb09ba31f34b5abdd419952845a68643d4d8df96e9a68643d4d8df96e9a68643d4d8df96e9ba029cd346349f36c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d0246b8ae4e81acb9fa143114829cf33ca746b8ae4e81acb9fa46b8ae4e81acb9fae992ad2cc54b8aa82351fe598a5894ec615ab5228c34303140e3fb81381f3591fed88e598638463b2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eabc6c679871524fbf7ddc1bb7967f85dcbbd0ca7a45a8813cec5b922a937441bee49f5ff0743e01b943fdc85926d4eb0def5a18441d6f5f811652c8d9bab9891ff60768b50dd5e12c36a95541ff881623729cd2db18e763082465d5650ed2fd5c1685428d00dc691fa9e82118971f2ed6494d66450ac1e729244734ef48a8669163de19da9ed218fe23de19da9ed218fe2da5ad592e16af4d7e45e6a89df371af55cc35db011f710a1998c47e34a42c61a; _vis_opt_exp_74_exclude=1; _vis_opt_exp_71_exclude=1; _vis_opt_exp_69_exclude=1; v=1527504434; nps_sleep=1; sx=H4sIAAAAAAACA53PUW7CMAyA4bv4mQcndcHtbTYnBMkBb%2FXaVKDenTJp0p65wPfrf8BAOFzmPNVZBBs5o6ApFYPxAQuM0E3f%2BJPz2iVUNhZWdSVmc1FuBQ6QYQx9PPUc%2Bxi2A8T1g76W83W9XZ0EtTibtqL6HkkvcpbAHd5iUkLWgu5SmjX%2FI2v4rCWJ1DvRPqFY0Hy39jY2b%2F%2FIAbn7JZeUZUp1rkcjdTHCpk5vkTEMtG1PS1Phf00BAAA%3D; isHeartsEnabled=0; _nfh=3cef57cbf431e38a89658bc44a326af9; _gat_UA-2546784-1=1",
           u"referer": url,
           u"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36",
           u"x-requested-with": "XMLHttpRequest"}
    pr_id = url.split("_")[-1]
    similars = requests.get("https://www.avito.ru/js/similar/" + pr_id + "?offset=0", headers=headers)
    sim_urls = []
    for i, json_part in enumerate(similars.json()['similarsJson']):
        sim_urls.append([url, i, "https://www.avito.ru" + json_part['url']])
    return sim_urls

In [24]:
#TODO Попробовать загрузить похожие и информацию об 1 продукте с авито
get_product_info_avito("https://www.avito.ru/moskva/akvarium/faraon_dlya_akvariuma_869897086")

('https://www.avito.ru/moskva/akvarium/faraon_dlya_akvariuma_869897086',
 'Объявление на Авито - Фараон для аквариума',
 'Статуя фараона для аквариума. Высота 22 см.')

In [ ]:
#Создадим список проксей и проверим их доступность

from user_agent import generate_user_agent
from time import sleep
import numpy as np
import requests

proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]

for i in range(len(proxies_list)):
    sleep(np.random.random())
    assert 200 == requests.get('https://yandex.ru', 
                               headers={'User-Agent': generate_user_agent()},
                               proxies=proxies_list[i]).status_code
print ('all_proxies_are_worked!\n\n')

In [ ]:
def get_proxy():
    return proxies_list[np.random.randint(0, len(proxies_list))]

#TODO сделаем декоратор, который будет генерировать случайную задержку на исполнение функции
def delay_decorator(func):
    def func_delay(*args, **kwargs):
        sleep(np.random.random())
        return func(*args, **kwargs)
    return func_delay

#TODO Модернизируем функцию, которая парсит похожие. Будем генерировать user-agent, не будем передавать cookies
#используем случайный прокси сервер. Обернем в декоратор задержки
def get_smilar_avito(url):
    headers = {u"accept": u"*/*", u"accept-encoding": u"gzip, deflate, br",
           u"accept-language": u"ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
           u"cookie": u"u=26m3i302.1fc6nq6.flumvzf17n; buyer_location_id=637640; abp=1; _ga=GA1.2.1386374517.1519811174; _vwo_uuid_v2=D5AF40B1213C5F49AEE8A45D8584418ED|06e85eceee6cdc4810096a7268f54acb; cto_lwid=85c9fbb1-8918-4cc4-a91d-2a757134e844; _vis_opt_s=1%7C; _vis_opt_test_cookie=1; _ym_uid=1519811174428571120; _vwo_uuid=D5AF40B1213C5F49AEE8A45D8584418ED; __gads=ID=c16722fa56933ba1:T=1519811174:S=ALNI_MYLAba7KfXbX-Amt7ypPSVfyyTdzA; _vis_opt_exp_46_exclude=1; __utma=99926606.1386374517.1519811174.1521126280.1521126280.1; __utmc=99926606; __utmz=99926606.1521126280.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); dfp_group=52; _vwo_ds=3%3Aa_0%2Ct_0%3A0%241526386623%3A47.11855471%3A%3A%3A18_0%2C12_0; sessid=bbc4979ea3347bd02e9b3ed7349589c7.1527492578; _gid=GA1.2.91958680.1527492579; _ym_isad=1; f=5.673c10cb09ba31f34b5abdd419952845a68643d4d8df96e9a68643d4d8df96e9a68643d4d8df96e9ba029cd346349f36c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d02c1e8912fd5a48d0246b8ae4e81acb9fa143114829cf33ca746b8ae4e81acb9fa46b8ae4e81acb9fae992ad2cc54b8aa82351fe598a5894ec615ab5228c34303140e3fb81381f3591fed88e598638463b2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eab2da10fb74cac1eabc6c679871524fbf7ddc1bb7967f85dcbbd0ca7a45a8813cec5b922a937441bee49f5ff0743e01b943fdc85926d4eb0def5a18441d6f5f811652c8d9bab9891ff60768b50dd5e12c36a95541ff881623729cd2db18e763082465d5650ed2fd5c1685428d00dc691fa9e82118971f2ed6494d66450ac1e729244734ef48a8669163de19da9ed218fe23de19da9ed218fe2da5ad592e16af4d7e45e6a89df371af55cc35db011f710a1998c47e34a42c61a; _vis_opt_exp_74_exclude=1; _vis_opt_exp_71_exclude=1; _vis_opt_exp_69_exclude=1; v=1527504434; nps_sleep=1; sx=H4sIAAAAAAACA53PUW7CMAyA4bv4mQcndcHtbTYnBMkBb%2FXaVKDenTJp0p65wPfrf8BAOFzmPNVZBBs5o6ApFYPxAQuM0E3f%2BJPz2iVUNhZWdSVmc1FuBQ6QYQx9PPUc%2Bxi2A8T1g76W83W9XZ0EtTibtqL6HkkvcpbAHd5iUkLWgu5SmjX%2FI2v4rCWJ1DvRPqFY0Hy39jY2b%2F%2FIAbn7JZeUZUp1rkcjdTHCpk5vkTEMtG1PS1Phf00BAAA%3D; isHeartsEnabled=0; _nfh=3cef57cbf431e38a89658bc44a326af9; _gat_UA-2546784-1=1",
           u"referer": url,
           u"user-agent": generate_user_agent(),
           u"x-requested-with": "XMLHttpRequest"}
    pr_id = url.split("_")[-1]
    similars = requests.get("https://www.avito.ru/js/similar/" + pr_id + "?offset=0", headers=headers, proxies_list = get_proxy())
    sim_urls = []
    for i, json_part in enumerate(similars.json()['similarsJson']):
        sim_urls.append([url, i, "https://www.avito.ru" + json_part['url']])
    return sim_urls

In [25]:
#Скачаем и разархивируем драйвер для гугл хрома https://chromedriver.storage.googleapis.com/index.html?path=2.43/
#Или firefox https://github.com/mozilla/geckodriver/releases

import pandas as pd
import selenium

from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException

from time import sleep

In [27]:
!ls

chromedriver  geckodriver.log  Scrapping.ipynb
geckodriver   regression       YH.xlsx


In [28]:
browser = Firefox(executable_path="../lecture_4/geckodriver")
browser.get('https://www.google.com/')

search_form = browser.find_element_by_name('q')
search_form.send_keys("в контакте")
search_form.send_keys(Keys.ENTER)

In [ ]:
#TODO Загрузить эксельку в pd.DataFrame
data = 
href = {}
data.head()

In [ ]:
name = data.loc[0, 'Short Name']

browser = Chrome(executable_path="chromedriver")

browser.get('https://finance.yahoo.com/quote/BAB?p=BAB&.tsrc=fin-srch-v1')
search_form = browser.find_element_by_xpath('''//*[@id="fin-srch-assist"]/input''')
search_form.send_keys(name)
search_form.send_keys(Keys.ENTER)
time.sleep(2)


sleep(2)
target_place = browser.find_element_by_xpath('''//*[@id="lookup-page"]/section/div/div/div/div/table/tbody/tr[1]/td[1]/a''')
target_place.click()

sleep(2)
history_place = browser.find_element_by_xpath('''//*[@id="quote-nav"]/ul/li[9]/a/span''')
history_place.click()
        
sleep(2)
dates = browser.find_element_by_xpath('''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/span/input''')
dates.click()

sleep(1)
max_ts = browser.find_element_by_xpath('''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/div[1]/span[8]/span''')
max_ts.click()

sleep(1)
done = browser.find_element_by_xpath('''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/div[3]/button[1]/span''')
done.click()

sleep(1)
apply = browser.find_element_by_xpath('''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button/span''')
apply.click()
result = browser.find_element_by_xpath('''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a''')
href[name] = result.get_attribute("href")

In [ ]:
#TODO Завернуть в функцию, которая парсит ShortName из data. Формируем dict href

In [30]:
#TODO Давайте спарсим выдачу Яндекса (1 страницу) на запрос по вашему Имени Фамилии
browser = Firefox(executable_path="../lecture_4/geckodriver")
browser.get('https://www.yandex.ru/')

search_form = browser.find_element_by_xpath('''//*[@id="text"]''')
search_form.send_keys("бебебе")
search_form.send_keys(Keys.ENTER)